In [126]:
import torch
import cv2
import numpy as np
from PIL import ImageGrab
import math
from pynput.mouse import Controller
import pyautogui

In [5]:
model = torch.hub.load('./', 'custom', path='./runs/train/exp/weights/best.pt', source='local')

YOLOv5 🚀 a8efbd2 Python-3.9.13 torch-1.13.0 CPU

Fusing layers... 
Model summary: 157 layers, 7099126 parameters, 0 gradients, 16.0 GFLOPs
Adding AutoShape... 


In [66]:
# 置信度阀值
model.conf = 0.25
# 自动混合精度推理
model.amp = True
# 一个box里多个标签
model.multi_label = True
# 指定labels number ,训练用的yaml里定义的每个label number对应的标签
model.classes = 0

In [76]:
class Point():
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
class Line(Point):
    def __init__(self, x1, y1, x2, y2):
        super().__init__(x1, y1, x2, y2)

    def getlen(self):
        changdu = math.sqrt(math.pow((self.x1 - self.x2), 2) + math.pow((self.y1 - self.y2), 2))
        return changdu

In [67]:
im = '../datasets/word/images/train/14XS.jpg'
# im = ImageGrab.grab()
results = model(im)
# results.pandas().xyxy[0].to_json(orient="records")
results.pandas().xyxy[0]

# results.print()
# results.show()

,xmin,ymin,xmax,ymax,confidence,class,name
0,36.78376,4.036248,54.430607,25.243761,0.711635,0,four


In [138]:
# 过滤模型
## 坐标
xmins = results.pandas().xyxy[0]['xmin']
ymins = results.pandas().xyxy[0]['ymin']
xmaxs = results.pandas().xyxy[0]['xmax']
ymaxs = results.pandas().xyxy[0]['ymax']

## 信任度
confidences = results.pandas().xyxy[0]['confidence']
name = results.pandas().xyxy[0]['name']

class_list = results.pandas().xyxy[0]['class']
# print(class_list)
# print(xmins)
# print(ymins)
# print(xmaxs)
# print(ymaxs)
# print(confidences)
# print(name)

newlist = []
for xmin, ymin, xmax, ymax, classitem, conf in zip(xmins, ymins, xmaxs, ymaxs, class_list, confidences):
    if classitem == 0 and conf > 0.5:
        newlist.append([int(xmin), int(ymin), int(xmax), int(ymax), conf])

# print(newlist)

# 中心坐标
for listItem in newlist:
    cdList = []
    xyList = []
    # print(listItem)
    xindex = int(listItem[2] - (listItem[2] - listItem[0]) / 2)
    yindex = int(listItem[3] - (listItem[3] - listItem[1]) / 2)
    mouseModal = Controller()
    x, y = mouseModal.position
    # print(x,y)
    L1 = Line(x, y, xindex, yindex)
    cdList.append(int(L1.getlen()))
    xyList.append([xindex, yindex, listItem[0], listItem[1], listItem[2], listItem[3]])

print(cdList)
print(xyList)
minCD = min(cdList)
print(minCD)

[857]
[[45, 14, 36, 4, 54, 25]]
857


In [139]:
game_width = 3712
game_height = 1120
for cdItem, xyItem in zip(cdList, xyList):
   print(cdItem)
   print(xyItem)
   # 2秒钟鼠标移动坐标为100,100位置  绝对移动
   # pyautogui.moveTo(100, 100,2)
   # pyautogui.moveTo(x=100, y=100,duration=2, tween=pyautogui.linear)
   # win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, int(xyItem[0] - game_width // 2),int(xyItem[1] - (game_height - (xyItem[3] - xyItem[5])) // 2 + 17), 0, 0)
   print(int(xyItem[0] - game_width // 2))
   print(int(xyItem[1] - (game_height - (xyItem[3] - xyItem[5])) // 2))
   x = int(xyItem[0] - game_width // 2)
   y = int(xyItem[1] - (game_height - (xyItem[3] - xyItem[5])) // 2)
   pyautogui.moveTo(x=x, y=y, duration=2, tween=pyautogui.linear)

857
[45, 14, 36, 4, 54, 25]
-1811
-556


In [8]:
from mss.darwin import MSS as mss
import matplotlib.pyplot as plt
sct = mss()

In [34]:
# 可以看每个显示器分辨率来判断显示器编号
print(sct.monitors[0])
print(sct.monitors[1])
print(sct.monitors[2])

{'left': -1920, 'top': 0, 'width': 3712, 'height': 1120}
{'left': 0, 'top': 0, 'width': 1792, 'height': 1120}
{'left': -1920, 'top': 0, 'width': 1920, 'height': 1080}


In [26]:
# 选中副显示器
monitor_number = 2
# 可以看每个显示器分辨率来判断显示器编号
# sct_img = sct.monitors[2]
# print(sct_img)
mon = sct.monitors[monitor_number]
print(mon["top"])
monitor = {
        "top": mon["top"] + 100,  # 100px from the top
        "left": mon["left"] + 100,  # 100px from the left
        "width": 1920,
        "height": 1080,
        "mon": monitor_number,
    }

0


In [16]:
import time
from pynput.mouse import Controller
# import mouse

In [ ]:
while True:
    mouseModal = Controller()
    x, y = mouseModal.position
    print(x, y)
    time.sleep(1)

In [31]:
import pyautogui
# 获取当前屏幕分辨率
screenWidth, screenHeight = pyautogui.size()
# 获取当前鼠标位置
currentMouseX, currentMouseY = pyautogui.position()

# 2秒钟鼠标移动坐标为100,100位置  绝对移动
#pyautogui.moveTo(100, 100,2)
# pyautogui.moveTo(x=100, y=100,duration=2, tween=pyautogui.linear)
#鼠标移到屏幕中央。
pyautogui.moveTo(screenWidth / 2, screenHeight / 2)

# 鼠标左击一次
# pyautogui.click()